In [1]:
from src.agent.planner import run_dq_check

result = run_dq_check("Check for duplicates in Snowflake table CUSTOMERS")

c:\Users\CihanGirayOner\Desktop\git_repos\llm_dq_agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\CihanGirayOner\Desktop\git_repos\llm_dq_agent\src\agent\planner.py:29: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: ca



> Entering new AgentExecutor chain...

Invoking: `check_dataset_duplicates` with `{'dataset_id': 'CUSTOMERS', 'connector_type': 'snowflake'}`


--- Loading data for: CUSTOMERS using SNOWFLAKE connector ---
✓ Connected to Snowflake: AGENT_LLM_READ.PUBLIC
Executing query: SELECT * FROM CUSTOMERS
✓ Loaded 6 rows from Snowflake
✓ Disconnected from Snowflake
{'dataset_id': 'CUSTOMERS', 'duplicate_qty': 2, 'status': 'failure'}The duplicate check for the Snowflake table `CUSTOMERS` has been completed. There are 2 duplicate rows found, but the operation encountered a failure. You may need to investigate further or retry the process.

> Finished chain.


In [2]:
# Display the result
print("Agent Output:")
print(result['output'])

Agent Output:
The duplicate check for the Snowflake table `CUSTOMERS` has been completed. There are 2 duplicate rows found, but the operation encountered a failure. You may need to investigate further or retry the process.
